In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#создаем датафреймы по событиям и сабмишнам
events_data = pd.read_csv('event_data_train.csv')
submissions_data = pd.read_csv('submissions_data_train.csv')

In [4]:
#добавляем в сабмишн датасет столбцы где переводим таймстемп в дату и день
submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
submissions_data['day'] = submissions_data.date.dt.date
submissions_data.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49,2016-11-11
3,31972,1478852164,correct,15853,2016-11-11 08:16:04,2016-11-11
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03,2015-06-15


In [7]:
events_data.day.max()

datetime.date(2018, 5, 19)

In [6]:
#аналогично для ивентс_дата
events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')
events_data['day'] = events_data.date.dt.date
events_data.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [5]:
#Реструктурируем events_data. В результате имеем датафрейм с инфой
# о том сколько каждый пользователь сделал discovered	passed	started_attempt	viewed
users_events_data = events_data.pivot_table(index='user_id',
                        columns='action',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()
users_events_data.head()

action,user_id,discovered,passed,started_attempt,viewed
0,1,1,0,0,1
1,2,9,9,2,10
2,3,91,87,30,192
3,5,11,11,4,12
4,7,1,1,0,1


In [6]:
#аналогично для сабмитов
users_scores = submissions_data.pivot_table(index='user_id',
                        columns='submission_status',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()
users_scores.head()

submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1


In [7]:
#создаем серию из events_data_test[['user_id', 'day', 'timestamp']]
#где оставляем уникальные сочетания user_id и day
#группируем по user_id и оставляем timestamp'ы последнего в дне события
#находим разницу между timestamp'ми. np.absolute на случай если разность между timestamp'ми будет отрицальная
#получаем np.array с разницой времени между событиями в каждый из дней
gap_data = events_data[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day']). \
    groupby('user_id')['timestamp'].apply(list). \
apply(np.diff).values

In [8]:
#преобразуем в серию
gap_data = pd.Series(np.concatenate(gap_data, axis=0))

In [9]:
#преобразуем timestamp в дни
gap_data = gap_data / (24 * 60 * 60)

In [10]:
#В users_data хранится время последнего действия для каждого пользователя
users_data = events_data.groupby('user_id', as_index=False).agg({'timestamp' : 'max'}).rename(columns ={'timestamp' :
                                                                                                        'last_timestamp'})

In [11]:
# now - время начала курса, drop_out_threshold - таймстемп 30 дней (считаем, что если 30 дней не заходил, то больше и не зайдет)
now = 1526772811
drop_out_threshold = 30 * 24 * 60 *60

In [12]:
# создание колонки is_gone_user True - выбыл, False - не выбыл
users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threshold

In [13]:
#Наращиваем users_data путем добавления инфы о users_scores
users_data = users_data.merge(users_scores, on='user_id', how='outer')

In [14]:
users_data = users_data.fillna(0)

In [15]:
#Наращиваем users_data путем добавления инфы о users_events_data
users_data = users_data.merge(users_events_data, how='outer')

In [16]:
# считаем сколько уникальных дней посещений у пользователя
users_days = events_data.groupby('user_id').day.nunique().to_frame().reset_index()

In [17]:
# Наращиваем users_data
users_data = users_data.merge(users_days, how='outer')

In [18]:
# считаем, что юзер прошел курс если набрал всего 170 баллов
users_data['passed_course'] = users_data.passed > 170

In [19]:
#B user_min_time хранится timestamp первого действия
user_min_time = events_data.groupby('user_id', as_index=False). \
    agg({'timestamp' : 'min'}).rename({'timestamp' : 'min_timestamp'}, axis=1)

In [20]:
#Добавляем инфу о времени первого действия
users_data = users_data.merge(user_min_time, how='outer')

In [148]:
# создаем dataframe event_data_train в котором будет инфа о первых n днях пользователя
event_data_train = pd.DataFrame()

In [21]:
#В user_time собираем сразу id и таймстемп события
events_data['user_time'] = events_data.user_id.map(str) + '_' + events_data.timestamp.map(str)

In [22]:
events_data.head()

,step_id,timestamp,action,user_id,date,day,user_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895


In [24]:
# порог 2-х дней
learning_time_threshold = 2 * 24 * 60 *60
learning_time_threshold

172800

In [25]:
#время порога двух дней относительно первого события
user_learning_time_threshold = user_min_time.user_id.map(str) + '_' + (user_min_time.min_timestamp + learning_time_threshold).map(str)

In [27]:
user_learning_time_threshold.head()

0    1_1473000264
1    2_1514556164
2    3_1434531276
3    5_1466329609
4    7_1521807460
dtype: object

In [28]:
#добавляем в dataframe
user_min_time['user_learning_time_threshold'] = user_learning_time_threshold

In [29]:
#наращиваем events_data
events_data = events_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how='outer')

In [30]:
events_data.head()

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,31981,1472827464,viewed,1,2016-09-02 14:44:24,2016-09-02,1_1472827464,1_1473000264
1,31981,1472827464,discovered,1,2016-09-02 14:44:24,2016-09-02,1_1472827464,1_1473000264
2,32815,1514383364,discovered,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164
3,32815,1514383364,viewed,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164
4,32815,1514383364,passed,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164


In [31]:
events_data.shape

(3480703, 8)

In [32]:
#фильтруем events_data, что в events_data_train попадают только первые два дня активности юзера
events_data_train = events_data[events_data.user_time <= events_data.user_learning_time_threshold]

In [33]:
events_data_train.head()

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,31981,1472827464,viewed,1,2016-09-02 14:44:24,2016-09-02,1_1472827464,1_1473000264
1,31981,1472827464,discovered,1,2016-09-02 14:44:24,2016-09-02,1_1472827464,1_1473000264
2,32815,1514383364,discovered,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164
3,32815,1514383364,viewed,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164
4,32815,1514383364,passed,2,2017-12-27 14:02:44,2017-12-27,2_1514383364,2_1514556164


In [34]:
events_data_train.groupby('user_id').day.nunique().max()

np.int64(3)

In [35]:
#Аналогично для submissions_data
submissions_data['user_time'] = submissions_data.user_id.map(str) + '_' + submissions_data.timestamp.map(str)
submissions_data = submissions_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how='outer')
submissions_data_train = submissions_data[submissions_data.user_time <= submissions_data.user_learning_time_threshold]
submissions_data_train.groupby('user_id').day.nunique().max()

np.int64(3)

In [36]:
#формируем датафрейм X
X = submissions_data_train.groupby('user_id').day.nunique().to_frame().reset_index() \
.rename(columns={'day' : 'days'})

In [37]:
X.head()

,user_id,days
0,2,1
1,3,1
2,8,1
3,14,1
4,16,2


In [38]:
#считаем сколько затраено степов
steps_tried = submissions_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index() \
.rename(columns={'step_id': 'steps_tried'})

In [40]:
steps_tried.head()

,user_id,steps_tried
0,2,2
1,3,4
2,8,11
3,14,1
4,16,18


In [41]:
#наращиваем X
X = X.merge(steps_tried, on='user_id', how='outer')

In [42]:
#наращиваем X путем реструктуризации submissions_data_test
X = X.merge(submissions_data_train.pivot_table(index='user_id',
                        columns='submission_status',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index())

In [43]:
X['correct_ratio'] = X.correct / (X.correct + X.wrong)

In [44]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio
0,2,1,2,2,0,1.000000
1,3,1,4,4,4,0.500000
2,8,1,11,9,21,0.300000
3,14,1,1,0,1,0.000000
4,16,2,18,18,23,0.439024


In [45]:
# наращиваем X путем реструктуризации events_data_test
X = X.merge(events_data_train.pivot_table(index='user_id',
                        columns='action',
                        values='step_id',
                        aggfunc='count',
                       fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')

In [46]:
X = X.fillna(0)

In [47]:
# наращиваем X 
X = X.merge(users_data[['user_id', 'passed_course', 'is_gone_user']], how='outer')

In [51]:
# отбрасываем пользователей которые прямо сейчас проходят курс
X = X[~((X.is_gone_user == False) & (X.passed_course == False))]

In [52]:
X.groupby(['passed_course', 'is_gone_user']).user_id.count()

passed_course  is_gone_user
False          True            16560
True           False             141
               True             1284
Name: user_id, dtype: int64

In [54]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,1,0.0,0.0,0.0,0.0,0.0,1,False,True
1,2,1.0,2.0,2.0,0.0,1.0,9,False,True
2,3,1.0,4.0,4.0,4.0,0.5,20,False,True
3,5,0.0,0.0,0.0,0.0,0.0,1,False,True
4,7,0.0,0.0,0.0,0.0,0.0,1,False,True


In [55]:
# формируем серию целевых значений
y = X.passed_course.map(int)

In [56]:
#отбрасываем лишнее
X = X.drop(['passed_course', 'is_gone_user'], axis=1)

In [57]:
#user_id кидаем в индекс датафрейма, а потом его дропаем с датафрейма
X = X.set_index(X.user_id)
X = X.drop(['user_id'], axis = 1)

In [58]:
X.head()

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1
2,1.0,2.0,2.0,0.0,1.0,9
3,1.0,4.0,4.0,4.0,0.5,20
5,0.0,0.0,0.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,1


In [59]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: passed_course, dtype: int64

In [61]:
#This is where the fun begins)
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [62]:
#сплитуем X, y на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [66]:
min_samples_split = range(round(0.001 * len(y)), round(0.01 * len(y)), 10)
min_samples_leaf = range(round(0.001 * len(y)), round(0.01 * len(y)), 10)

In [67]:
# Дерево решений
clf_DT = DecisionTreeClassifier()
params_DTC  = {'criterion':['entropy', 'gini'], 'max_depth' : range(1,5), 
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf }

In [68]:
#делаем сетку параметров модели
best_clf_DT = GridSearchCV(clf_DT, params_DTC, cv=6, n_jobs=-1)
# обучаем модели
best_clf_DT.fit(X_train, y_train)
# лучшую модель пихаем в clf
clf_DT = best_clf.best_estimator_

In [70]:
clf_DT

,criterion,'entropy'
,splitter,'best'
,max_depth,4
,min_samples_split,18
,min_samples_leaf,58
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [74]:
#считаем вероятности событий для пользователей по дереву
y_predicted_prob = clf.predict_proba(X_test)

# выводим гистограммы вероятностей 0 и 1
# pd.Series(y_predicted_prob[:, 0]).hist()
# pd.Series(y_predicted_prob[:, 1]).hist()

In [75]:
#считаем roc_auc_score
roc_auc_score(y_test, y_predicted_prob[:,1])

0.8382198703279937

In [76]:
from sklearn.linear_model import LogisticRegression

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [78]:
#Смотрим логистическую регрессию
clf_LR = LogisticRegression()
#по сути выбираем модель солвера
params_LR =  {'solver': ['lbfgs','liblinear','newton-cg', 'newton-cholesky', 'sag','saga'], 'max_iter': [10000]}

In [79]:
best_clf_LR = GridSearchCV(clf_LR, params_LR, cv=6, n_jobs=-1)
best_clf_LR.fit(X_train, y_train)
clf_LR = best_clf_LR.best_estimator_

In [80]:
clf_LR

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


In [81]:
y_predicted_prob = clf.predict_proba(X_test)
roc_auc_score(y_test, y_predicted_prob[:,1])

0.8221093948639715

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [88]:
#случайный лес
clf_RF = RandomForestClassifier()

In [89]:
params_rf = {'n_estimators': range(20, 201, 10), 'criterion': ['gini', 'entropy'], 'max_depth': range(1,10), 
            'min_samples_split': range(18, 60, 2), 'min_samples_leaf': range(18, 60, 2), 'bootstrap': [True],
            }

In [95]:
#Раз лес случайный, то поиск параметров случайный (шутка)
#экономим время. Если времени много можно все перебрать
#результат по рандому +- одинаковый
best_clf_RF = RandomizedSearchCV(clf_RF, params_rf, cv=6, n_jobs=-1)
best_clf_RF.fit(X_train, y_train)
clf_RF = best_clf_RF.best_estimator_

In [96]:
y_predicted_prob = clf.predict_proba(X_test)
roc_auc_score(y_test, y_predicted_prob[:,1])

0.8319640435630139

In [92]:
clf_RF

,n_estimators,120
,criterion,'entropy'
,max_depth,9
,min_samples_split,42
,min_samples_leaf,22
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [266]:
#сохраняем модель случайного леса в pkl файл
import pickle
filename = 'trained_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(clf_RF, file)